In [1]:
# PATHS
DATA_TRAIN_PATH = "./../data/raw/train.csv"
DATA_TEST_PATH = "./../data/raw/test.csv" 
OUTPUT_PATH = './../results/predictions/oct2_paola.csv'

In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./../src/')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from implementations import *
from utils import *
from pipeline import * 
from proj1_helpers import * 

In [4]:
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [5]:
# Simple division
# TODO: Cross validation

X_train, X_test, y_train, y_test = split_data(tX, y, 0.1, myseed=1)
X_train, X_test = preprocessing(X_train, X_test)


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

# Baseline

In [15]:
clf = LogisticRegression(max_iter = 700)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

/usr/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

        -1.0       0.78      0.86      0.82    147877
         1.0       0.67      0.54      0.60     77123

    accuracy                           0.75    225000
   macro avg       0.73      0.70      0.71    225000
weighted avg       0.74      0.75      0.74    225000



In [18]:
clf = LogisticRegression(max_iter = 700)
parameters = {'penalty':['l1','l2','elasticnet'], 
              'C':[0.001, 0.1, 0.5, 1, 2, 10],
             'l1_ratio':[0.25,0.5,0.75]}
gsc = GridSearchCV(clf, parameters,
                    scoring = 'f1', 
                    cv = 5, 
                    verbose = 1,
                    n_jobs = -1)

gsc.fit(tX, y)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed: 13.4min finished
/usr/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn("l1_ratio parameter is only used when penalty is "
/usr/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_r

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=700), n_jobs=-1,
             param_grid={'C': [0.001, 0.1, 0.5, 1, 2, 10],
                         'l1_ratio': [0.25, 0.5, 0.75],
                         'penalty': ['l1', 'l2', 'elasticnet']},
             scoring='f1', verbose=1)

In [21]:
best_params =gsc.best_params_
best_params

{'C': 0.1, 'l1_ratio': 0.25, 'penalty': 'l2'}

In [22]:
gsc.best_score_

0.5694741426405251

In [23]:
clf = LogisticRegression(max_iter = 700, C=0.1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        -1.0       0.78      0.87      0.82    147877
         1.0       0.67      0.53      0.59     77123

    accuracy                           0.75    225000
   macro avg       0.73      0.70      0.71    225000
weighted avg       0.74      0.75      0.74    225000



In [46]:
ws = []
losses = []

#Gridsearch for lambda_
w = np.zeros(X_train.shape[1])
gammas = [0.01,0.1,0.2]
lambdas_ = [0.0001,0.001,0.01, 0.05,0.1, 0.2, 0.5, 1]
for l in lambdas_:
    for g in gammas:
        w, loss = reg_logistic_regression(y_train, X_train, l,w,100,g)
        ws.append(w)
        losses.append(loss)


Current iteration=0, loss=17328.67951399868


./../src/implementations.py:50: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1 + np.exp(-t))
./../src/implementations.py:56: RuntimeWarning: divide by zero encountered in log
  loss = y.T.dot(np.log(pred)) + (1 - y).T.dot(np.log(1 - pred))


Current iteration=0, loss=nan
Current iteration=0, loss=nan
Current iteration=0, loss=nan
Current iteration=0, loss=nan
Current iteration=0, loss=nan
Current iteration=0, loss=nan
Current iteration=0, loss=nan
Current iteration=0, loss=nan
Current iteration=0, loss=nan
Current iteration=0, loss=nan
Current iteration=0, loss=nan
Current iteration=0, loss=nan
Current iteration=0, loss=nan
Current iteration=0, loss=nan
Current iteration=0, loss=nan
Current iteration=0, loss=nan


KeyboardInterrupt: 

In [56]:
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

# Treat test day the same way we treat traim
_, tX_test = preprocessing(tX, tX_test)

weights = ws[0]
y_pred = predict_labels(weights, tX_test,1)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

## Missing
* Cross validation for hyperparameters 